## Enderscopy stage and lights demo

This notebook shows how to connect and drive an enderscope stage from Python.

In [1]:
import time
from enderscope import Stage, Panel, Enderlights
from serial_utils import serial_ports

In [2]:
# list available serial ports
ports = serial_ports()
print (ports)

['/dev/tty.Bluetooth-Incoming-Port', '/dev/tty.usbserial-142130', '/dev/tty.usbmodem142301']


In [56]:
# connect to enderscope and create a stage object
s = Stage(ports[1], 115200)

In [57]:
# create a stage control panel
p = Panel(s)

GridspecLayout(children=(Button(description='Up', layout=Layout(grid_area='widget001', height='auto', width='a…

Output()

In [5]:
p.recorded_positions

[(50.0, 120.0, 0.0), None, None, None, None, None]

In [7]:
# iterate over manually recorded positions
for i in range(5):
    for pos in p.recorded_positions:
        s.move_position(pos)
        s.finish_moves()
        

In [14]:
# home the stage for positionning reproducibility
s.home()
# move to a safe distance along the z axis, axis is x, y or z, distances are in millimeters
s.move_axis('z',10)

In [25]:
# read the stage position
s.get_position()

(-18.0, -2.0, 10.0)

In [26]:
# position can be retrieved as a dict
s.get_position(dict=True)

{'X': -18.0, 'Y': -2.0, 'Z': 10.0, 'E': 0.0}

In [27]:
# move to (x,y(,z)) absolute coordinate
s.move_absolute(50,120)

In [ ]:
# debug view the actual G-Code that was sent
s.move_absolute(20,30, debug=True)

In [10]:
# you can head towards standard directions, distances are always > 0
s.move_towards('east',70)

In [28]:
# you can move by some amount, relative to the current position
s.move_relative(0,0,5)

In [29]:
for i in range(15):
    s.move_relative(-1,0,0)

In [30]:
# a position is a tuple of length 2 or 3 to which you can move
pos = (20,20)
s.move_position(pos)

In [10]:
p = (40,40,10)
s.move_position(p)

In [17]:
# sending specific gcode to read device parameters
s.write_code(f"M503", debug=True)

echo:  G21    ; Units in mm (mm)
echo:  M149 C ; Units in Celsius

echo:; Filament settings: Disabled
echo:  M200 S0 D1.75
echo:; Steps per unit:
echo: M92 X80.00 Y80.00 Z400.00 E93.00
echo:; Maximum feedrates (units/s):
echo:  M203 X300.00 Y300.00 Z5.00 E25.00
echo:; Maximum Acceleration (units/s2):
echo:  M201 X3000.00 Y3000.00 Z100.00 E10000.00
echo:; Acceleration (units/s2): P<print_accel> R<retract_accel> T<travel_accel>
echo:  M204 P3000.00 R3000.00 T3000.00
echo:; Advanced: B<min_segment_time_us> S<min_feedrate> T<min_travel_feedrate> X<max_x_jerk> Y<max_y_jerk> Z<max_z_jerk> E<max_e_jerk>
echo:  M205 B20000.00 S0.00 T0.00 X10.00 Y10.00 Z0.30 E5.00
echo:; Home offset:
echo:  M206 X0.00 Y0.00 Z0.00
echo:; Material heatup parameters:
echo:  M145 S0 H200.00 B60.00 F0
echo:  M145 S1 H240.00 B80.00 F0
echo:; PID settings:
echo:  M301 P19.98 I0.88 D50.29
echo:  M304 P462.10 I85.47 D624.59
echo:; Power-Loss Recovery:
echo:  M413 S1
echo:; UI Language:
echo:  M414 S1
M503


'ok\n'

In [ ]:
s.write_code(f"M203 Z50") # increase z axis speed to 50
s.move_axis('z',40) # go up fast
s.write_code(f"M203 Z5") # restore z axis speed to 5
s.move_axis('z',-40) # go down slowly

In [15]:
l = Enderlights(ports[2])

In [16]:
# querrying LED state
done = l.write_code(f"?\n")

RGB:100;10;30


In [17]:
# setting LED colors
l.color(100,0,0)
l.green(10)
l.blue(30)

In [18]:
# blinking LEDs ;-)
for i in range(10):
    l.shutter(True)
    time.sleep(0.05)
    l.red(100)
    l.shutter(False)
    time.sleep(0.05)

In [22]:
# create an array of positions to perform a raster scan
# import time
import numpy as np
origin = [50,120] # coordinates of first position
steps = [25,-25] # x and y offsets to next item
grid = np.array(list((x,y) for y in range(3) for x in range(4))) * steps + origin
for p in grid:
    s.move_position(p)
    s.finish_moves() 
    l.shutter(True)
    time.sleep(0.5)
    l.shutter(False)

s.move_position(origin) # move back to origin


In [68]:
# arbitrary curve from gcode
# create curve in inkscape, export as svg, import in slicer, export gcode
import os
def read_g_code(fname):
    fpath = os.path.join(".", fname)
    with open(fpath, "r") as text_file:
        data = text_file.read()
    return data
path = read_g_code("tests/path2.gcode")

In [69]:
for i in range(1,50):
    bit = path.split('\n')[i]
    bit = bit[0:bit.index('E')-1]
    s.write_code(bit, debug=True)

G1 X34.751 Y59.885
G1 X29.685 Y58.899
G1 X27.082 Y58.291
G1 X24.533 Y57.604
G1 X22.121 Y56.839
G1 X19.932 Y55.998
G1 X18.023 Y55.073
G1 X16.475 Y54.064
G1 X15.861 Y53.529
G1 X15.368 Y52.974
G1 X15.005 Y52.399
G1 X14.769 Y51.77
G1 X14.593 Y50.62
G1 X14.636 Y49.518
G1 X14.883 Y48.465
G1 X15.293 Y47.499
G1 X15.841 Y46.583
G1 X16.499 Y45.707
G1 X18.018 Y44.041
G1 X19.629 Y42.412
G1 X21.087 Y40.751
G1 X21.675 Y39.899
G1 X22.146 Y38.998
G1 X22.463 Y38.051
G1 X22.598 Y37.047
G1 X22.523 Y36.038
G1 X22.256 Y35.087
G1 X21.818 Y34.168
G1 X21.256 Y33.305
G1 X19.876 Y31.656
G1 X18.339 Y30.017
G1 X16.905 Y28.32
G1 X16.301 Y27.418
G1 X15.817 Y26.471
G1 X15.474 Y25.442
G1 X15.323 Y24.393
G1 X15.376 Y23.237
G1 X15.68 Y21.966
G1 X16.223 Y20.68
G1 X17.009 Y19.349
G1 X17.992 Y18.041
G1 X19.151 Y16.762
G1 X20.486 Y15.506
G1 X21.946 Y14.318
G1 X23.539 Y13.191
G1 X25.211 Y12.161
G1 X26.973 Y11.228
G1 X28.77 Y10.423
G1 X30.615 Y9.748
